In [ ]:
!pip install --upgrade torch

Requirement already up-to-date: torch in /usr/local/lib/python3.7/dist-packages (1.8.1+cu101)


In [ ]:
# %%capture
!pip install transformers

     |████████████████████████████████| 2.3MB 26.3MB/s 
     |████████████████████████████████| 3.3MB 38.7MB/s 
     |████████████████████████████████| 901kB 49.0MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from tqdm import tqdm
import torch
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModel
from transformers import EarlyStoppingCallback

In [ ]:
# Downloading the datasets - kannada

!gdown https://drive.google.com/uc?id=1PLkXHY7OSmjJW5Jv2O2F4O8wFTU9DkO5  #test

!gdown https://drive.google.com/uc?id=1IbFbXfihA0Poeuju5mGVFRGcO-smUbFI  #dev

!gdown https://drive.google.com/uc?id=19lV8Y7hrq0OWHkTTGlHoJDBkXZUUHf7c  #train

# Downloading the datasets - tamil

!gdown https://drive.google.com/uc?id=1v6NWGPvo7rGGtqokD3sRJq-QWdMI_ebW  #dev

!gdown https://drive.google.com/uc?id=1mkIIz49JYr8VRPu-JKitX1Nn5UPLmTDg  #train

!gdown https://drive.google.com/uc?id=1y84b5fB3zOBHyVt8ELZr0RlBmxbdVdjm  #test

# Downloading the datasets - malayalam

!gdown https://drive.google.com/uc?id=1ZKoB9TQ34jntTr3CbrF_kNTdL5hQMUHe  #train

!gdown https://drive.google.com/uc?id=1Nylsy3DjjMK63lRy5Bx2s6ckSp5WdAWu  #dev

!gdown https://drive.google.com/uc?id=1-XOS0p_ZetvMDstZ4MXgCD4kuBTQd_M6  #test

Downloading...
From: https://drive.google.com/uc?id=1PLkXHY7OSmjJW5Jv2O2F4O8wFTU9DkO5
To: /content/kannada_offensive_test_with_labels.xlsx
100% 36.6k/36.6k [00:00<00:00, 14.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IbFbXfihA0Poeuju5mGVFRGcO-smUbFI
To: /content/kannada_offensive_dev.xlsx
100% 35.8k/35.8k [00:00<00:00, 58.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=19lV8Y7hrq0OWHkTTGlHoJDBkXZUUHf7c
To: /content/kannada_offensive_train.xlsx
100% 250k/250k [00:00<00:00, 8.05MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v6NWGPvo7rGGtqokD3sRJq-QWdMI_ebW
To: /content/tamil_offensive_full_dev.xlsx
100% 221k/221k [00:00<00:00, 14.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mkIIz49JYr8VRPu-JKitX1Nn5UPLmTDg
To: /content/tamil_offensive_full_train.xlsx
100% 1.72M/1.72M [00:00<00:00, 27.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1y84b5fB3zOBHyVt8ELZr0RlBmxbdVdjm
To: /content/tamil_offensive_full_test_with_labels.xlsx
100%

In [ ]:
# For now only choose tamil [1]

lang = input('Choose language: 1 for tamil, 2 for malayalam, 3 for kannada: ')

class_list = ['Not_offensive',
 'Offensive_Targeted_Insult_Group',
 'Offensive_Targeted_Insult_Individual',
 'Offensive_Targeted_Insult_Other',
 'Offensive_Untargetede']


train_file_name = None
dev_file_name = None

if lang == '1': 
  lang = 'Tamil'
  class_list.append('not-Tamil')
  train_file_name = 'tamil_offensive_full_train.xlsx'
  dev_file_name = 'tamil_offensive_full_dev.xlsx'
  test_file_name = 'tamil_offensive_full_test_with_labels.xlsx'
  label_to_index = {'Not_offensive':0,
                  'Offensive_Targeted_Insult_Group':1,
                  'Offensive_Targeted_Insult_Individual':2,
                  'Offensive_Targeted_Insult_Other':3,
                  'Offensive_Untargetede':4,
                  'not-Tamil':5}

elif lang == '2': 
  lang = 'malayalam'
  # As it contains only 5 classes ('Offensive_Targeted_Insult_Other' is not present)
  class_list.append('not-malayalam')
  class_list.remove('Offensive_Targeted_Insult_Other')

  train_file_name = 'mal_full_offensive_train.xlsx'
  dev_file_name = 'mal_full_offensive_dev.xlsx'
  test_file_name = 'mal_full_offensive_test_with_labels.xlsx'
  label_to_index = {'Not_offensive':0,
                  'Offensive_Targeted_Insult_Group':1,
                  'Offensive_Targeted_Insult_Individual':2,
                  'Offensive_Untargetede':3,
                  'not-malayalam':4}

else:
  lang = 'Kannada'
  class_list.append('not-Kannada')
  train_file_name = 'kannada_offensive_train.xlsx'
  dev_file_name = 'kannada_offensive_dev.xlsx'
  test_file_name = 'kannada_offensive_test_with_labels.xlsx'
  label_to_index = {'Not_offensive':0,
                  'Offensive_Targeted_Insult_Group':1,
                  'Offensive_Targeted_Insult_Individual':2,
                  'Offensive_Targeted_Insult_Other':3,
                  'Offensive_Untargetede':4,
                  'not-Kannada':5}

Choose language: 1 for tamil, 2 for malayalam, 3 for kannada: 2


In [ ]:
class_list

['Not_offensive',
 'Offensive_Targeted_Insult_Group',
 'Offensive_Targeted_Insult_Individual',
 'Offensive_Untargetede',
 'not-malayalam']

In [ ]:
train_df = pd.read_excel(train_file_name, header=None)
train_df.columns = ['Text', 'Label']
train_df = train_df.dropna().drop_duplicates().reset_index(drop=True, inplace=False)

dev_df = pd.read_excel(dev_file_name, header=None)
dev_df.columns = ['Text', 'Label']

test_df = pd.read_excel(test_file_name, header=None, engine='openpyxl')
test_df.columns = ['Text', 'Label']

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 1e-05
tokenizer = AutoTokenizer.from_pretrained('google/muril-base-cased')

## Defining the classes

In [ ]:
class MuRILModel(torch.nn.Module):
  def __init__(self, n_classes):
    super(MuRILModel, self).__init__()
    self.l1 = AutoModel.from_pretrained("google/muril-base-cased")
    self.pre_classifier = torch.nn.Linear(768, 768)
    self.dropout = torch.nn.Dropout(0.3)
    self.classifier = torch.nn.Linear(768, n_classes)

  def forward(self, input_ids, attention_mask, token_type_ids):
    output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    pooler = self.pre_classifier(output_1['pooler_output'])
    pooler = torch.nn.ReLU()(pooler)
    pooler = self.dropout(pooler)
    output = self.classifier(pooler)
    return output

class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.sent = dataframe.Text
    self.targets = dataframe.Label
    self.max_len = max_len

  def __len__(self):
    return len(self.sent)

  def __getitem__(self, index):
    sent = str(self.sent[index])
    sent = " ".join(sent.split())

    inputs = self.tokenizer.encode_plus(sent,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        pad_to_max_length=True,
                                        return_token_type_ids=True
                                      )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]


    return {
      'ids': torch.tensor(ids, dtype=torch.long),
      'mask': torch.tensor(mask, dtype=torch.long),
      'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
      'targets': torch.tensor(self.targets[index], dtype=torch.long)
    }

class EarlyStopping():
  """
  Early stopping to stop the training when the loss does not improve after
  certain epochs.
  """
  def __init__(self, patience=5, min_delta=0):
    """
    :param patience: how many epochs to wait before stopping when loss is
            not improving
    :param min_delta: minimum difference between new loss and old loss for
            new loss to be considered as an improvement
    """
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.best_loss = None
    self.early_stop = False
  
  def __call__(self, val_loss):
    if self.best_loss == None:
      self.best_loss = val_loss
    elif self.best_loss - val_loss > self.min_delta:
      self.best_loss = val_loss
    elif self.best_loss - val_loss < self.min_delta:
      self.counter += 1
      print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
      if self.counter >= self.patience:
        print('INFO: Early stopping')
        self.early_stop = True

In [ ]:
train_df['Label'] = train_df['Label'].apply(lambda x: label_to_index[x])
dev_df['Label'] = dev_df['Label'].apply(lambda x: label_to_index[x]) 

In [ ]:
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(dev_df, tokenizer, MAX_LEN)

In [ ]:
from torch.utils.data import DataLoader

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

model = MuRILModel(len(class_list))
model = model.float()
model.to(device)
print('Loaded!')

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loaded!


In [ ]:
# calculate class weights
inverse_weights = np.array(train_df['Label'].value_counts().sort_index())
class_weights = np.sum(inverse_weights) / inverse_weights

class_weights = (torch.tensor(class_weights, dtype=torch.float)).cuda()

In [ ]:
loss_function = torch.nn.CrossEntropyLoss(weight = class_weights)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
  n_correct = (preds==targets).sum().item()
  return n_correct

In [ ]:
losslsit = []
acclist = []

In [ ]:
def train(epoch):
  tr_loss = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0
  model.train()
  
  for _,data in tqdm(enumerate(training_loader, 0)):
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)

    outputs = model(ids, mask, token_type_ids)
    loss = loss_function(outputs, targets)
    print(loss)
    tr_loss += loss.item()
    big_val, big_idx = torch.max(outputs.data, dim=1)
    n_correct += calcuate_accuracy(big_idx, targets)
    nb_tr_steps += 1
    nb_tr_examples+=targets.size(0)
    #print(targets)
    #print(big_idx)
    optimizer.zero_grad()
    loss.backward()
    # # When using GPU
    optimizer.step()
  
  print(f'The Total Accuracy for Epoch {epoch} : {(n_correct*100)/nb_tr_examples}')
  epoch_loss = tr_loss/nb_tr_steps
  epoch_accu = (n_correct*100)/nb_tr_examples
  losslsit.append(float(epoch_loss))
  acclist.append(float(epoch_accu))
  print(f"Training Loss Epoch: {epoch_loss}")
  print(f"Training Accuracy Epoch: {epoch_accu}")
  torch.save(model.state_dict(), f'/content/{lang}_{epoch}.pt', )

In [ ]:
EPOCHS=5

for epoch in range(EPOCHS):
    train(epoch)
    

print('training Complete...')
print('saving model...')

torch.save(model.state_dict(), f'/content/{lang}_final.pt', )

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tensor(1.5936, device='cuda:0', grad_fn=<NllLossBackward>)


1it [00:00,  1.30it/s]

tensor(1.5936, device='cuda:0', grad_fn=<NllLossBackward>)


2it [00:01,  1.42it/s]

...

tensor(1.0446, device='cuda:0', grad_fn=<NllLossBackward>)


730it [07:45,  1.57it/s]

tensor(0.1484, device='cuda:0', grad_fn=<NllLossBackward>)


731it [07:45,  1.57it/s]


The Total Accuracy for Epoch 4 : 89.8392062949025
Training Loss Epoch: 0.6758251605267482
Training Accuracy Epoch: 89.8392062949025
training Complete...
saving model...


In [ ]:
from google.colab import files
files.download('_final.pt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>